## Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
from sklearn.datasets import load_boston
import pandas as pd
from pathlib import Path

In [2]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [3]:
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt import UtilityFunction
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [4]:
%run SVR_LS_Library_percent.ipynb 

## Load data

In [5]:
rng = check_random_state(0)
boston = load_boston()
perm = rng.permutation(boston.target.size)
boston.data = boston.data[perm]
boston.target = boston.target[perm]
# 
Y = boston.target
X = boston.data

In [13]:
X = pd.read_csv("new_boston.csv")
X = X.drop(["Unnamed: 0"], axis = 1)

In [14]:
# X = pd.DataFrame(X)
# X = pd.get_dummies(X, columns = ["3"], prefix = "dum")

In [15]:
#split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state = 5)

#normalize
scaler = MaxAbsScaler().fit(x_train)
X_train = scaler.transform(x_train); X_test = scaler.transform(x_test)

scaler1 = MaxAbsScaler().fit(y_train.reshape(-1, 1))
Y_train = scaler1.transform(y_train.reshape(-1, 1)).reshape(-1)
Y_test = scaler1.transform(y_test.reshape(-1, 1)).reshape(-1)

## Optimization function

In [16]:
def opt_bas(C, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVR_PerLs(**hyperparameters).fit(X_train, Y_train)

    y_pred = model.predict(X_test)
    
    # rescale
#     y_pred = scaler1.inverse_transform(predict.reshape(-1, 1)).reshape(-1)
    
    # get score
    mape = np.mean(np.abs((Y_test - y_pred)/Y_test))*100
    
    return -mape

## Run optimization

In [17]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [21]:
# Bounded region of parameter space
pbounds = {'C': (19.9, 50), 'gamma': (0.3, 1)}

# Domain reduction function
# bounds_transformer = SequentialDomainReductionTransformer()

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
#                                  bounds_transformer=bounds_transformer
                                )

# keep data
log_path = Path().resolve() / "Logs" / "New_Boston_correjido.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [19]:
optimizer.probe(params={"C": 131.6135372548593, "gamma": 0.1926931038470609}, lazy = False)
optimizer.probe(params={"C": 4, "gamma": 2}, lazy = False)
optimizer.probe(params={"C": 7.628874476243877, "gamma": 1.1659271563361087}, lazy = False)

In [22]:
optimizer.maximize(init_points=30, n_iter=600)

In [ ]:
# from bayes_opt.util import load_logs


# new_optimizer = BayesianOptimization(
#     f=black_box_function,
#     pbounds={"x": (-2, 2), "y": (-2, 2)},
#     verbose=2,
#     random_state=7,
# )
# log_path = Path().resolve() / "Logs" / "test2.json"
# # New optimizer is loaded with previously seen points
# load_logs(new_optimizer, logs=["./logs.json"]);